# 第4章 再帰

再帰とは関数定義の中で自分自身を呼び出すこと  
無限に処理するものでなければ基底部(これ以上分解できない値)を定義し、そこで再帰するのを終了させる

再帰的に定義したmaximum関数

復習  
`=>`は型クラス(インターフェースとして使える)  
Ord型クラスはなんらかの順序付けするための型クラス

In [5]:
maximum' :: (Ord a) => [a] -> a
-- パターンマッチ 空リストか要素が一つの場合の条件分岐
maximum' [] = error "empty list"
maximum' [x] = x
-- リストの先頭を取得して、残りのリストを再帰的にmaximum'に渡す
maximum' (x:xs) = max x (maximum' xs)

maximum' [2,5,1]

5

上記再帰の流れの詳細  
maximum'に`[2,5,1]`を渡す  
`2`と`[5,1]`に分けて`max 2 (maximum' [5,1])`になる  
`maximum' [5,1]`は`max 5 (maximum' [1])`で`maximum' [1]`は`1`を返すので`max 5 1`で出力は5になる  
`max 2 (maximum' [5,1])`は`max 2 5`になって結果として5を出力する


### replicate

intと値をとり、int分、値を繰り返したリストを作る `replicate 3 5`は`[5,5,5]`を返す  

メモ
初回 `5 : replicate' 2 5`  
`replicate' 2 5`は`5: replicate' 1 5`  
`replicate' 1 5`は`5: replicate' 0 5`  
`replicate' 0 5`は`[]`  

逆順でみてくと  
`5: []`  
`5 : 5: []`  
`5 : 5 : 5: []`  
で出力が`[5,5,5]`になる


In [7]:
replicate' :: Int -> a -> [a]
replicate' n x
    | n <= 0 =[]
    | otherwise = x : replicate' (n-1) x

replicate' 3 5

[5,5,5]

復習  
リストの表現方法
`[1,2,3]` = `1 : 2 : 3 : []`

In [14]:
1 : 2 : 3 : []

Line 1: Use list literal
Found:
1 : 2 : 3 : []
Why not:
[1, 2, 3]

[1,2,3]

### take

指定されたリストから指定された数分の先頭からの要素のリストを返す  
`take 2 [1,2,3]`は`[1,2]`を返す

In [15]:
take' :: Int -> [a] -> [a]

-- 第一引数が0以下の場合 第二引数の値がいずれかであろうと同じ値を返すので_を使用
take' n _
    | n <= 0 = []
    
-- 第二引数が空リストの場合　第一引数の値がいずれかであろうと同じ値を返すので_を使用
take' _ [] = []

-- 第二引数のリストをheadとtaileに分けてhead : (n-1した値とtaileのリストをtake'に渡す)
take' n (x:xs) = x : take' (n-1) xs

take' 2 [1,2,3]


[1,2]

### reverse

リストを逆順にする  
リストの先頭の要素を取り出して `xs ++ [x]`を再帰関数の引数に渡す  

`reverse' [1,2,3]`の場合は  
`reverse' [2,3] ++ [1]`  
`reverse' [3] ++ [2] ++ [1]`  
`[] ++ [3] ++ [2] ++ [1]`  

注意するのは`reverse' [2,3] ++ [1]`は`reverse'`の引数は`[2,3]`であり`++ [1]`は別物であること

In [16]:
reverse' :: [a] -> [a]
reverse' [] = []
reverse' (x:xs) = reverse' xs ++ [x]

reverse' [1,2,3]

[3,2,1]

要素数が一つのリストを`x:xs`で分けるとどうなるのか

In [4]:
(x:xs) = [1]
x
xs

1

[]

In [5]:
[] ++ [3] ++ [2] ++ [1]

[3,2,1]

### repeat  
要素を受け取り、その要素の無限リストを作る  
基底部がないので無限リストになる

In [7]:
repeat' :: a-> [a]
repeat' x = x : repeat' x

### zip  
二つのリストを引数に、これらを合わせる  
`zip [1,2,3] [7,8]`は`[(1,7), (2,8)]`になる

基底部に第一位引数が空リストになった場合、第二引数が空リストになった場合を定義する
各引数のheadを組み合わせてタプルを作り、残りのtaileは再帰関数の引数として渡す

In [9]:
zip' :: [a] -> [b] -> [(a,b)]
zip' _ []  = []
zip' [] _ = []
zip' (x:xs) (y:ys) = (x,y): zip' xs ys

zip' [1,2,3] [7,8]

[(1,7),(2,8)]

### elem  
値とリスト受け取り、その値がリストに含まれているかを調べる  
基底部に第二引数が空リストになったらFalseになるように設定  
ガードを使い第一引数とheadが同一であればTrue
それ以外は再帰関数に第一引数の値とtaileを渡す

In [12]:
elem' :: (Eq a) => a -> [a] -> Bool
elem' _ [] = False
elem' a (x:xs)
    | a == x = True
    | otherwise = a `elem'` xs

elem' 1 [1,2,3]
elem' 'b' "saterait"

True

False

## クイックソート

数などのソート処理  
最初の要素を選択し、その要素以下の物を左に、その要素より大きい物を右に移動させる  
最終的に空リストになるように何回も行う  

`[5,1,9,4,6]`  
`[1,4] ++ [5] ++ [9,6]`

`[1,4]`は先頭の1より小さい値がないので左には空リストを  
`[] ++ [1] ++ [4]`  
`[4]`は4より小さい値、大きい値がないので下記のようになる  
`[] ++ [4] ++ []`  

In [14]:
quicksort :: (Ord a) => [a] -> [a]
quicksort [] = []
quicksort (x:xs) = 
    let
        -- リスト内包表記 先頭の要素の値未満のリスト
        smallerOrEqual = [a | a <- xs, a <= x]
        -- 先頭の要素より大きい値のリスト
        larger = [a | a <- xs, a > x]
    in quicksort smallerOrEqual ++ [x] ++ quicksort larger

quicksort [5,1,9,4,6]

[1,4,5,6,9]

処理内容を追ってみる

`quicksort [5,1,9,4,6]`  
`quicksort [1,4] ++ [5] ++ quicksort [9,6]`  

`quicksort [1,4]`  
`quicksort [] ++ [1] ++ quicksort [4]`    
` [] ++ [1] ++ quicksort [4]`  
`[] ++ [1] ++ [] ++ [4] ++ []`  
`[1,4]`  

`quicksort [9,6]`    
`quicksort [6] ++ [9] ++ []`  
`[] ++ [6] ++ [] ++ [9] ++ []`  
`[6,9]`  

最初に戻って
`[1,4] ++ [5] ++ [6,9]`






再帰処理を考える時はまず基底部を定義するべし  
さらにとくべき問題を小さく分割するべし